# 对象定义

## 引入和定义

In [1]:
from typing import List, Tuple
from bitstring import BitArray, BitStream
import random
import numpy as np

# 训练数据，是一个二元组，前者表示输入数据，后者表示预期结果数据，形如：[([1,1],[2]),([1,2],[3])]
# 表示一组训练数据
OneGroupData = Tuple[List[float],List[float]]
# 表示多组训练数据
TrainData: List[OneGroupData] = None

## 公共函数

In [2]:
def RandomBitArray(length: int):
    res = []
    for i in range(length):
        res.append(random.choice('01'))
    return BitArray(bin=''.join(res))

## Man对象

个体

### 初始化函数

In [3]:
def Man_Init(self, cellsNum: int):
    # 神经元的数量
    self.cellsNum = cellsNum

    # 输入神经元
    self.inputCell = InputCell(self)
    # 输出神经元
    self.outputCell = OutputCell(self)

    # 神经元列表
    self.cells: List[Cell] = []
    # 初始化神经元列表
    self.cells.append(self.inputCell)
    self.cells.append(self.outputCell)
    # 增加普通神经元
    for i in range(self.cellsNum):
        self.cells.append(Cell(self))

    self.token = 0
    
    # 适应度 = 1 - 偏差率；偏差率 = Abs[（计算值 - 实际值）/ 实际值 ]
    self.fitness: float = None

### ComputeFitness

In [4]:
def ComputeFitness(self, trainData: TrainData) -> None:
    '''
    trainData：训练数据
    功能：计算并设置Man对象的适应度
    '''
    fitnessValues = list(map(self.ComputeFitness_OneGroup,trainData))
    res = np.mean(fitnessValues)
    self.fitness = res

#### ComputeFitness_OneGroup

In [5]:
def ComputeFitness_OneGroup(self, data: OneGroupData) -> float:
    '''
    计算单组训练数据的平均适应度
    返回值：平均适应度
    '''
    def 适应度(dst, bas):
        res = (dst - bas)/bas
        return 1 - abs(res)
    
    inputs = data[0]
    outputs = data[1]
    computeValues = self.Compute(inputs, len(outputs))
    fitnessValues = list(map(适应度, computeValues, outputs))
    res = np.mean(fitnessValues)
    return res

### Compute：计算单组输入的输出

In [6]:
def OneCompute(self):
    '''进行一次计算'''
    self.token += 1
    self.inputCell.compute(self.token)

def Compute(self, inputs: List[float], outputNum: int) -> List[float]:
    '''
    功能：输入一组数据，计算对应的输出数据
    inputs：一组输入数据
    outputNum：输出数据的数量
    返回：一组输出数据
    '''
    # 输入阶段
    for v in inputs:
        self.inputCell.value = v
        self.OneCompute()

    # 思考阶段，暂时先思考一次吧
    self.OneCompute()

    # 输出阶段
    res = []
    for i in range(outputNum):
        self.OneCompute()
        res.append(self.outputCell.value)

    return res

### Man：类定义

In [7]:
class Man:
    # 初始化函数
    __init__ = Man_Init
    # 计算适应度函数
    ComputeFitness = ComputeFitness
    # 计算单组训练数据的平均适应度
    ComputeFitness_OneGroup = ComputeFitness_OneGroup
    OneCompute = OneCompute
    # 输入一组数据，计算对应的输出数据
    Compute = Compute

## Cell对象

神经元

In [8]:
class Cell:
    def __init__(self, owner: Man):
        super().__init__()
        # 设置其所属的父对象
        self.owner = owner
        # 连接到自己的神经元有哪些，0表示没有连接，1表示有连接
        self.inCells: BitArray = None
        # 连接到自己的神经元的权重
        self.weights: List[float] = None
        self.bitWeights: BitArray = None
        # 自己连接到的神经元
        self.outCells: BitArray = None
        # 神经元自身的值
        self.value: float = None
        # 神经元自身的偏移量
        self.b: float = None
        # 计算令牌
        self.token = -1
        
        # 随机化
        self._randomParas()
    
    # 随机化相关参数
    def _randomParas(self):
        # 神经元的数量
        length = self.owner.cellsNum
        self.inCells = RandomBitArray(length)
        self.outCells = RandomBitArray(length)
        self.bitWeights = RandomBitArray(length*64)
        self.weights = BitStream(self.bitWeights).readlist(f'{length}*float:64')
        self.value = RandomBitArray(64).float
        self.b = RandomBitArray(64).float
    
    # 计算输入神经元的加权和
    def _computeSum(self) -> float:
        res = self.b
        for i,w in enumerate(self.inCells):
            # 如果没有连接，则不参与计算
            if w == 0: continue
            res += self.owner.cells[i].value * self.weights[i]
        return res
    
    def _IsContinue(self, token) -> bool:
        '''根据计算令牌，判断是否需要继续计算'''
        if self.token == token: return False
        self.token = token
        return True
    
    def _computeOthers(self, token):
        '''遍历它所连接的神经元'''
        for i,w in enumerate(self.outCells):
            if w == 0: continue
            self.owner.cells[i].compute(token)
    
    def compute(self, token: int) -> None:
        '''
        token：计算令牌用于防止重复计算
        '''
        if self._IsContinue(token)==False: return
        
        # 更新自身神经元的值
        _sum = self._computeSum()
        # 采用ReLU激励函数
        self.value = 0 if _sum<0 else _sum
        
        # 遍历它所连接的神经元
        self._computeOthers(token)

### InputCell对象

In [9]:
class  InputCell(Cell):
    def compute(self, token: int) -> None:
        if self._IsContinue(token)==False: return
        # 自身的值不需要更新
        # 遍历它所连接的神经元
        self._computeOthers(token)

### OutputCell对象

In [10]:
class  OutputCell(Cell):
    def compute(self, token: int) -> None:
        if self._IsContinue(token)==False: return
        # 更新自身神经元的值
        _sum = self._computeSum()
        # 最后这个可以不用激励函数了
        self.value = _sum
        # 遍历它所连接的神经元
        self._computeOthers(token)

## Ai对象

AI系统

In [11]:
class Ai:
    pass

# 测试代码

man = Man(1)
t1 = ([1,1],[2])
t2 = ([1,2],[3])
trainData = [t1,t2]
man.ComputeFitness(trainData)
res = man.fitness
print(res)